In [1]:
import pickle
import numpy as np
import pandas as pd

In [5]:
#load model
model = pickle.load(open('model_192600.sav','rb'))

In [6]:
df1 = pd.read_csv("../Dataset/weather_merged_2/add_feature_trafficData192600.csv", dtype = str)
df1.head(3)

,year,month,day,hour,REPORT_ID,avgMeasuredTime,vehicleCount,avgSpeed
0,2014,08,01,07,192600,72.0,40,60.0
1,2014,08,01,08,192600,81.41666666666667,235,53.583333333333336
2,2014,08,01,09,192600,79.16666666666667,234,56.0


In [18]:
def make_metrics(model):
    data = {
        'name': model.name ,
        'r2': model.r2 ,
        'rmse': model.rmse
    }
    # data['name'] = 'average R2 and sum RMSE'
    # data['r2'].append(np.mean(data['r2']))
    # data['rmse'].append(np.sum(data['rmse']))
    return pd.DataFrame(data)

In [12]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

In [13]:
class Model:
  def __init__(self, name, data, predict_features, test_size, ml_model):
    self.name = name
    self.data = data
    self.predict_features = predict_features
    self.is_trained = False
    self.test_size = test_size
    self.ml_model = ml_model
    self.do_things()

  def cal_rmse(self):
    self.rmse = mean_squared_error(self.ytest, self.ypredict, squared=False)
    return self.rmse

  def prequisite(self, test_size):
    self.features = [i for i in self.data.columns if i != self.predict_features]
    self.X = self.data[self.features].values
    self.y = self.data[self.predict_features].values
    self.Xtrain, self.Xtest, self.ytrain, self.ytest = train_test_split(self.X, self.y, test_size=test_size)
    return None

  def fit(self):
    self.is_trained = True
    self.ml_model.fit(self.Xtrain, self.ytrain)
    self.ypredict = self.ml_model.predict(self.Xtest)
    return self.ml_model

  def cal_r2_score(self):
    self.r2 = r2_score(self.ytest, self.ypredict)
    return self.r2

  def do_things(self) -> None:
    self.prequisite(self.test_size)
    self.fit()
    self.cal_rmse()
    self.cal_r2_score()
    return None

  def __repr__(self) -> str:
    if not self.is_trained:
      return f'<{self.name}> (is not trained yet)>'
    return f'<({self.name}: [R² Score: {self.r2}], [RMSE: {self.rmse}])>'

In [19]:

model = Model(
            ml_model=model,
            name=f'Dataset of report 192600',
            data=df1,
            predict_features='vehicleCount',
            test_size=1/4
        )
make_metrics(model)

TypeError: fit() takes 1 positional argument but 3 were given